In [2]:
import numpy as np
import pandas as pd

dataset = pd.read_csv('datasets/Data.csv')
print(dataset)
X = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]

print('done')

         AT      V       AP     RH      PE
0     14.96  41.76  1024.07  73.17  463.26
1     25.18  62.96  1020.04  59.08  444.37
2      5.11  39.40  1012.16  92.14  488.56
3     20.86  57.32  1010.24  76.64  446.48
4     10.82  37.50  1009.23  96.62  473.90
...     ...    ...      ...    ...     ...
9563  16.65  49.69  1014.01  91.00  460.03
9564  13.19  39.18  1023.67  66.78  469.62
9565  31.32  74.33  1012.92  36.48  429.57
9566  24.48  69.45  1013.86  62.39  435.74
9567  21.60  62.52  1017.23  67.87  453.28

[9568 rows x 5 columns]
done


In [ ]:
#split
#scale

# multiple
# polynomial
# svr
# decision_tree
# random_forest